In [3]:
# standard
import pandas as pd
import numpy as np
import random
import os
import math

# tf and keras
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers
from tensorflow.keras.layers import GlobalAveragePooling2D
from PIL import ImageFile

# sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# plots
# import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image, ImageOps
from pathlib import Path

In [4]:
# Set Variables
DATA_PATH = Path("./300x225")

In [5]:
# Read the image data
train_images = np.load(DATA_PATH / "train_images_aug.npy")
val_images = np.load(DATA_PATH / "val_images.npy")
test_images = np.load(DATA_PATH / "test_images.npy")

# Read the label data
train_labels = np.load(DATA_PATH / "train_labels_aug.npy")
val_labels = np.load(DATA_PATH / "val_labels.npy")
test_labels = np.load(DATA_PATH / "test_labels.npy")

# Read the  metadata
md_df = pd.read_csv(DATA_PATH / "metadata.csv")

### Feature Exploration

In [6]:
md_df.columns

Index(['Unnamed: 0', 'eventDate', 'year', 'month', 'day', 'habitat',
       'countryCode', 'scientificName', 'kingdom', 'phylum', 'class', 'order',
       'family', 'genus', 'specificEpithet', 'hasCoordinate', 'species',
       'iucnRedListCategory', 'substrate', 'latitude', 'longitude',
       'coorUncert', 'observationID', 'region', 'district', 'filename',
       'category_id', 'metaSubstrate', 'poisonous', 'elevation', 'landcover',
       'biogeographicalRegion', 'split', 'image_path', 'class_label',
       'class_idx'],
      dtype='object')

In [11]:
potential_features = ["habitat", "biogeographicalRegion", "elevation"]

In [19]:
# Set a random seed and clear back end
tf.keras.backend.clear_session()
tf.random.set_seed(1234)

# Get total number of classes
all_labels = np.concat([train_labels, val_labels, test_labels])
num_classes = len(set(all_labels))

# Add the biogeographical region from the metadata
bio_region = tf.keras.layers.Input(shape=(1,), dtype=tf.string, name="biogeographicalRegion")
bio_region_id = tf.keras.layers.StringLookup(
    vocabulary=list(md_df.biogeographicalRegion.unique()), output_mode="one_hot"
)(bio_region)

# Add the image data
pics = tf.keras.layers.Input(shape=(300, 225, 3), dtype=tf.float32, name="images")

# Convolutional Layer
conv = tf.keras.layers.Conv2D(3, kernel_size=4, padding="same", activation="relu")(pics)
# Pooling Layer
pooling = tf.keras.layers.MaxPool2D()(conv)
# Dropout Layer
dropout = tf.keras.layers.Dropout(0.25)(pooling)

# Concatenate all of the features
features = tf.keras.layers.Concatenate()([bio_region_id, pics])

# Multiclass layer
dense = tf.keras.layers.Dense(units=num_classes, activation="relu")(features)

model = tf.keras.Model(
    inputs=[bio_region],
    outputs=dense
)


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 9), (None, 300, 225, 3)]

In [16]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ biogeographicalRegion           │ (None, 1)              │             0 │
│ (InputLayer)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ string_lookup (StringLookup)    │ (None, 9)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate (Concatenate)       │ (None, 9)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 31)             │           310 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 310 (1.21 KB)

 Trainable params: 310 (1.21 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Set a random seed and clear back end
tf.keras.backend.clear_session()
tf.random.set_seed(1234)

# Get total number of classes
all_labels = np.concat([train_labels, val_labels, test_labels])
num_classes = len(set(all_labels))

model = tf.keras.Sequential([
    # Convolutional Layer
    tf.keras.layers.Conv2D(3, kernel_size=4, padding="same", activation="relu"),
    # Pooling Layer
    tf.keras.layers.MaxPool2D(),
    # Dropout Layer
    tf.keras.layers.Dropout(0.25),
    # Convolutional Layer
    tf.keras.layers.Conv2D(3, kernel_size=4, padding="same", activation="relu"),
    # Pooling Layer
    tf.keras.layers.MaxPool2D(),
    # Dropout Layer
    tf.keras.layers.Dropout(0.25),
    # Flattening
    tf.keras.layers.Flatten(),
    # Dense (Multiclassification Layer)
    tf.keras.layers.Dense(num_classes)
])

model.build(input_shape=(None, 300, 225, 3))
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 300, 225, 32)   │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 150, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 150, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 150, 112, 32)   │        16,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 75, 56, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 75, 56, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 134400)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 31)             │     4,166,431 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,184,415 (15.96 MB)

 Trainable params: 4,184,415 (15.96 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    verbose=1,
    patience=5,
    mode='max',
    restore_best_weights=True
)

In [12]:
history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels), callbacks=[early_stopping])

Epoch 1/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 263s 670ms/step - accuracy: 0.3876 - loss: 6.4255 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 2/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 259s 666ms/step - accuracy: 0.1349 - loss: 6.8807 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 3/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 273s 701ms/step - accuracy: 0.1349 - loss: 6.8807 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 4/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 280s 720ms/step - accuracy: 0.1349 - loss: 6.8807 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 5/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 285s 731ms/step - accuracy: 0.1349 - loss: 6.8807 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 6/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 312s 801ms/step - accuracy: 0.1349 - loss: 6.8807 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.


In [22]:
wee = model.evaluate(test_images, test_labels)

/Users/rkalafos/207/207-final-project/.venv/lib/python3.11/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['biogeographicalRegion']
Received: inputs=Tensor(shape=(None, 300, 225, 3))
  warnings.warn(msg)


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("functional_1/Cast:0", shape=(None, 300, 225, 3), dtype=string). Expected shape (None, 1), but input has incompatible shape (None, 300, 225, 3)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(None, 300, 225, 3), dtype=float32)
  • training=False
  • mask=None